**BBC - Liverpool**

In [ ]:
##BBC

In [14]:
import time
import re
import pandas as pd
from datetime import datetime

# Selenium imports
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By

# BeautifulSoup import
from bs4 import BeautifulSoup

# 1. Set up Microsoft Edge WebDriver
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")  # Run in headless mode
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2. Navigate to your target URL (replace with your actual news page URL)
url = "https://www.bbc.co.uk/sport/football/teams/liverpool"
driver.get(url)

# 3. Wait for JavaScript to load the page content
time.sleep(5)

# 4. Get the rendered HTML from Selenium
html = driver.page_source

# 5. Close the browser if no further interaction is needed
driver.quit()

# 6. Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# 7. Find all article containers (adjust the class if needed)
articles = soup.find_all("article", class_="ssrcss-1epmku9-ContentPost e6wdqbx1")

news_data = []

for article in articles:
    # --- Extract Date and Time from the <span> ---
    date_span = article.find("span")
    if date_span:
        # Get the full text in one line
        raw_date = date_span.get_text(separator=" ", strip=True)
        
        # Extract time (first occurrence of HH:MM)
        time_matches = re.findall(r"(\d{1,2}:\d{2})", raw_date)
        if time_matches:
            time_text = time_matches[0]
        else:
            time_text = "00:00"
        
        # Remove time occurrences to isolate potential date part
        date_part = re.sub(r"\d{1,2}:\d{2}", "", raw_date).strip()
        tokens = date_part.split()
        if len(tokens) >= 2:
            # Assume the first two tokens represent the day and month (e.g., "15 March")
            extracted_date_str = " ".join(tokens[:2])
            try:
                # Parse the extracted date string (assuming format "%d %B") and set default year 2025
                date_obj = datetime.strptime(extracted_date_str, "%d %B")
                # Format the date as M/D/YYYY; on Windows use %#m/%#d/%Y to avoid leading zeros.
                date_text = date_obj.replace(year=2025).strftime("%#m/%#d/%Y")
            except Exception as e:
                date_text = "3/17/2025"
        else:
            date_text = "3/17/2025"
    else:
        date_text = "3/17/2025"
        time_text = "00:00"
    
    # --- Extract Headline ---
    headline_tag = article.find("span", class_="ssrcss-189b1h2-HeadlineWrap e14e9ror2")
    if headline_tag:
        headline_text = headline_tag.get_text(strip=True)
    else:
        headline_text = "No Headline"
    
    # --- Extract Body Context (All Paragraphs) ---
    paragraphs = article.find_all("p", class_=re.compile("Paragraph"))
    if paragraphs:
        body_text = " ".join(p.get_text(strip=True) for p in paragraphs)
    else:
        body_text = "No Content"
    
    # Append the results: Date, Time, Headline, Body Context
    news_data.append([date_text, time_text, headline_text, body_text])

# 8. Convert to a DataFrame with columns: Date, Time, Headline, Body Context
df = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "Body Context"])

# 9. Display or save the DataFrame
print(df.head())

# Display the first few rows of the DataFrame
df.to_csv("Test.csv", index=False)

        Date   Time                                           Headline  \
0  3/17/2025  15:14  Is Salah's cup finals record his Achilles heel...   
1  3/17/2025  15:00  Liverpool 'can suffer' final defeat - Jamespub...   
2  3/17/2025  13:55  'We want to bounce back, and we will'published...   
3  3/17/2025  12:06  'Lack of quality depth must be addressed'publi...   
4  3/17/2025  09:55  'We played into their hands'published at 09:55...   

                                        Body Context  
0  Alongside his team-mates, it was a difficult d...  
1  Former Liverpool goalkeeper David James says L...  
2  Liverpool captain Virgil van Dijk on Instagram...  
3  In the space of a week, the Arne Slot machine ...  
4  Andy Robertson says Liverpool "didn't use the ...  


In [16]:
import time
import re
import pandas as pd
from datetime import datetime

# Selenium imports
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By

# BeautifulSoup import
from bs4 import BeautifulSoup

# 1. Set up Microsoft Edge WebDriver (once)
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")  # Run in headless mode
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

all_news_data = []  # Will store data from all pages

# 2. Loop through pages 1 to 40
for page in range(1, 41):
    # Build the URL based on the page number
    if page == 1:
        url = "https://www.bbc.co.uk/sport/football/teams/liverpool"
    else:
        url = f"https://www.bbc.co.uk/sport/football/teams/liverpool?page={page}"

    print(f"Scraping Page {page}: {url}")
    
    # 3. Navigate to the page
    driver.get(url)
    # Wait for JavaScript to load
    time.sleep(3)

    # 4. Get the rendered HTML from Selenium
    html = driver.page_source

    # 5. Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # 6. Find all article containers (Adjust class if needed)
    articles = soup.find_all("article", class_="ssrcss-1epmku9-ContentPost e6wdqbx1")

    for article in articles:
        # --- Extract Date and Time from the <span> ---
        date_span = article.find("span")
        if date_span:
            raw_date = date_span.get_text(separator=" ", strip=True)
            
            # Extract time (first occurrence of HH:MM)
            time_matches = re.findall(r"(\d{1,2}:\d{2})", raw_date)
            if time_matches:
                time_text = time_matches[0]
            else:
                time_text = "00:00"
            
            # Remove time occurrences to isolate potential date part
            date_part = re.sub(r"\d{1,2}:\d{2}", "", raw_date).strip()
            tokens = date_part.split()
            if len(tokens) >= 2:
                # Assume the first two tokens represent the day and month (e.g., "15 March")
                extracted_date_str = " ".join(tokens[:2])
                try:
                    # Parse the extracted date string (format "%d %B") with default year 2025
                    date_obj = datetime.strptime(extracted_date_str, "%d %B")
                    # Format as M/D/YYYY (on Windows use %#m/%#d/%Y to avoid leading zeros)
                    date_text = date_obj.replace(year=2025).strftime("%#m/%#d/%Y")
                except Exception:
                    date_text = "3/17/2025"
            else:
                date_text = "3/17/2025"
        else:
            date_text = "3/17/2025"
            time_text = "00:00"
        
        # --- Extract Headline ---
        headline_tag = article.find("span", class_="ssrcss-189b1h2-HeadlineWrap e14e9ror2")
        if headline_tag:
            headline_text = headline_tag.get_text(strip=True)
            # Remove everything from 'published at' onward (case-insensitive)
            lower_headline = headline_text.lower()
            idx = lower_headline.find("published at")
            if idx != -1:
                headline_text = headline_text[:idx].strip()
        else:
            headline_text = "No Headline"
        
        # --- Extract Body Context (All Paragraphs) ---
        paragraphs = article.find_all("p", class_=re.compile("Paragraph"))
        if paragraphs:
            body_text = " ".join(p.get_text(strip=True) for p in paragraphs)
        else:
            body_text = "No Content"
        
        # Append the results: Date, Time, Headline, Body Context
        all_news_data.append([date_text, time_text, headline_text, body_text])

# 7. After all pages are scraped, close the driver
driver.quit()

# 8. Convert to a DataFrame with columns: Date, Time, Headline, Body Context
df = pd.DataFrame(all_news_data, columns=["Date", "Time", "Headline", "Body Context"])

# 9. Save to CSV
df.to_csv("BBC_AllPages.csv", index=False)
print("Scraping complete! Data saved to AllPages.csv")

Scraping Page 1: https://www.bbc.co.uk/sport/football/teams/liverpool
Scraping Page 2: https://www.bbc.co.uk/sport/football/teams/liverpool?page=2
Scraping Page 3: https://www.bbc.co.uk/sport/football/teams/liverpool?page=3
Scraping Page 4: https://www.bbc.co.uk/sport/football/teams/liverpool?page=4
Scraping Page 5: https://www.bbc.co.uk/sport/football/teams/liverpool?page=5
Scraping Page 6: https://www.bbc.co.uk/sport/football/teams/liverpool?page=6
Scraping Page 7: https://www.bbc.co.uk/sport/football/teams/liverpool?page=7
Scraping Page 8: https://www.bbc.co.uk/sport/football/teams/liverpool?page=8
Scraping Page 9: https://www.bbc.co.uk/sport/football/teams/liverpool?page=9
Scraping Page 10: https://www.bbc.co.uk/sport/football/teams/liverpool?page=10
Scraping Page 11: https://www.bbc.co.uk/sport/football/teams/liverpool?page=11
Scraping Page 12: https://www.bbc.co.uk/sport/football/teams/liverpool?page=12
Scraping Page 13: https://www.bbc.co.uk/sport/football/teams/liverpool?page=1

In [ ]:
##SkySport

In [46]:
import time
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 1️⃣ **Set up Microsoft Edge WebDriver**
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
# Disable headless mode for debugging
# edge_options.add_argument("--headless")  # Remove this to see browser actions
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2️⃣ **Navigate to the target URL**
url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

# 3️⃣ **Wait for the page to fully load**
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

# 4️⃣ **Click "Load More" repeatedly until it disappears**
def click_load_more():
    while True:
        try:
            load_more_button = driver.find_element(By.CLASS_NAME, "ui-liveblog-button--load-more")
            
            if load_more_button.is_displayed():
                print("✅ Load More button found! Clicking...")

                # Ensure the button is in view
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
                time.sleep(1)  # Allow scrolling effect

                # Click using JavaScript (to avoid "not clickable" errors)
                driver.execute_script("arguments[0].click();", load_more_button)

                # Wait for new content to load
                time.sleep(5)

                # Ensure new elements are loaded before proceeding
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

            else:
                print("❌ Load More button no longer visible.")
                break
        
        except Exception as e:
            print(f"❌ Load More button NOT found or all content loaded: {e}")
            break  # Stop when button disappears

# Call function to load all news
click_load_more()

# 5️⃣ **Get the full rendered HTML after all content is loaded**
html = driver.page_source

# 6️⃣ **Close the browser**
driver.quit()

# 7️⃣ **Parse HTML with BeautifulSoup**
soup = BeautifulSoup(html, "html.parser")

# 8️⃣ **Find all articles inside both the pinned and main list containers**
articles = soup.select("div.ncpost-list-post")

news_data = []

for article in articles:
    # **Extract Headline**
    headline_tag = article.find("h2", class_="ncpost-title")
    headline_text = headline_tag.get_text(strip=True) if headline_tag else "No Headline"

    # **Extract Date & Time from <time>**
    time_tag = article.find("time", class_="ncpost-timestamp")
    if time_tag and time_tag.has_attr("datetime"):
        datetime_str = time_tag["datetime"]
        date_obj = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%fZ")

        # Fix for Windows compatibility
        try:
            date_text = date_obj.strftime("%#m/%#d/%Y")  # Works on Windows
        except ValueError:
            date_text = date_obj.strftime("%-m/%-d/%Y")  # Works on Linux/macOS
            
        time_text = date_obj.strftime("%H:%M")  # Format as HH:MM
    else:
        date_text, time_text = "Unknown", "00:00"

    # **Extract Body Context (All Paragraphs)**
    paragraphs = article.select("div.ncpost-content p")
    body_text = " ".join(p.get_text(strip=True) for p in paragraphs) if paragraphs else "No Content"

    # **Append results**
    news_data.append([date_text, time_text, headline_text, body_text])

# 9️⃣ **Convert to DataFrame**
dfs = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "Body Context"])

# 🔟 **Save or Display the DataFrame**
print(dfs.head())
dfs.to_csv("skysports_liverpool_full.csv", index=False)

print("✅ Scraping completed successfully!")

✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More 

In [31]:
import time  
from selenium import webdriver  
from selenium.webdriver.edge.service import Service as EdgeService  
from selenium.webdriver.edge.options import Options as EdgeOptions  
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  

# 1️⃣ **Set up Microsoft Edge WebDriver**
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")  # Run in headless mode (remove this if you want to see the browser)
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2️⃣ **Navigate to your target URL**
url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

# 3️⃣ **Wait for the page to load**
time.sleep(5)  # Allow JavaScript to load (you can adjust this)

try:
    load_more_button = driver.find_element(By.CLASS_NAME, "ui-liveblog-button--load-more")
    print("✅ Load More button found!")
except:
    print("❌ Load More button NOT found!")

✅ Load More button found!


In [32]:
import time  
from selenium import webdriver  
from selenium.webdriver.edge.service import Service as EdgeService  
from selenium.webdriver.edge.options import Options as EdgeOptions  
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  

# 1️⃣ **Set up Microsoft Edge WebDriver**
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")  # Run in headless mode (remove this if you want to see the browser)
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2️⃣ **Navigate to your target URL**
url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

# 3️⃣ **Wait for the page to load**
time.sleep(5)  # Allow JavaScript to load (you can adjust this)

if load_more_button:
    driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
    time.sleep(2)  # Let page adjust
    load_more_button.click()
    print("✅ Load More button clicked after scrolling.")

NoSuchElementException: Message: no such element: element not found
  (Session info: MicrosoftEdge=134.0.3124.72); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60E82E735+13397]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF60EABB1B4+2060276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF60EA12338+1368440]
	(No symbol) [0x00007FF60E5CBC86]
	(No symbol) [0x00007FF60E5CAB7F]
	(No symbol) [0x00007FF60E5CC321]
	(No symbol) [0x00007FF60E5C065E]
	(No symbol) [0x00007FF60E5CC352]
	(No symbol) [0x00007FF60E5C065E]
	(No symbol) [0x00007FF60E5BEA75]
	(No symbol) [0x00007FF60E5C27DD]
	(No symbol) [0x00007FF60E642574]
	(No symbol) [0x00007FF60E62425A]
	(No symbol) [0x00007FF60E5F964D]
	(No symbol) [0x00007FF60E641C00]
	(No symbol) [0x00007FF60E623FC3]
	(No symbol) [0x00007FF60E5F8B96]
	(No symbol) [0x00007FF60E5F7E50]
	(No

In [41]:
import time  
from selenium import webdriver  
from selenium.webdriver.edge.service import Service as EdgeService  
from selenium.webdriver.edge.options import Options as EdgeOptions  
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  

# 1️⃣ **Set up Microsoft Edge WebDriver**
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2️⃣ **Navigate to your target URL**
url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

# Wait for the page to load
time.sleep(5)

try:
    load_more_button = driver.find_element(By.CSS_SELECTOR, "button.ui-liveblog-button--load-more")
    print("✅ Load More button found!")

    # Scroll into view
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
    time.sleep(2)  # Allow page to adjust

    # Force click using JavaScript
    driver.execute_script("arguments[0].click();", load_more_button)
    print("✅ Load More button clicked with JavaScript!")

except Exception as e:
    print(f"❌ Load More button NOT found or not clickable: {e}")

✅ Load More button found!
✅ Load More button clicked with JavaScript!


In [45]:
import time
import re
import pandas as pd
from datetime import datetime

# Selenium imports
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 1️⃣ **Set up Microsoft Edge WebDriver**
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
# **Disable headless mode to observe behavior**
# edge_options.add_argument("--headless")  # Remove this for visual debugging
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2️⃣ **Navigate to your target URL**
url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

# 3️⃣ **Wait for JavaScript to load the initial page content**
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

# 4️⃣ **Click Load More Button Once**
try:
    load_more_button = driver.find_element(By.CLASS_NAME, "ui-liveblog-button--load-more")
    if load_more_button.is_displayed():
        print("✅ Load More button found! Clicking once for testing...")

        # Ensure button is in view
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        time.sleep(1)  # Allow scrolling effect
        
        # Click using JavaScript
        driver.execute_script("arguments[0].click();", load_more_button)

        # Allow time for new content to load
        time.sleep(5)  # Adjust based on your connection speed

        # Ensure new content appears before proceeding
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

        print("✅ Load More button clicked successfully!")

    else:
        print("❌ Load More button found but not visible.")
except Exception as e:
    print(f"❌ Load More button NOT found or not clickable: {e}")

# 5️⃣ **Get the full rendered HTML after clicking Load More**
html = driver.page_source

# 6️⃣ **Close the browser**
driver.quit()

# 7️⃣ **Parse HTML with BeautifulSoup**
soup = BeautifulSoup(html, "html.parser")

# 8️⃣ **Find all articles inside both the pinned and main list containers**
articles = soup.select("div.ncpost-list-post")

news_data = []

for article in articles:
    # **Extract Headline**
    headline_tag = article.find("h2", class_="ncpost-title")
    headline_text = headline_tag.get_text(strip=True) if headline_tag else "No Headline"

    # **Extract Date & Time from <time>**
    time_tag = article.find("time", class_="ncpost-timestamp")
    if time_tag and time_tag.has_attr("datetime"):
        datetime_str = time_tag["datetime"]
        date_obj = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%fZ")

        # Fix for Windows compatibility
        try:
            date_text = date_obj.strftime("%#m/%#d/%Y")  # Works on Windows
        except ValueError:
            date_text = date_obj.strftime("%-m/%-d/%Y")  # Works on Linux/macOS
            
        time_text = date_obj.strftime("%H:%M")  # Format as HH:MM
    else:
        date_text, time_text = "Unknown", "00:00"

    # **Extract Body Context (All Paragraphs)**
    paragraphs = article.select("div.ncpost-content p")
    body_text = " ".join(p.get_text(strip=True) for p in paragraphs) if paragraphs else "No Content"

    # **Append results**
    news_data.append([date_text, time_text, headline_text, body_text])

# 9️⃣ **Convert to DataFrame**
dfs = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "Body Context"])

# 🔟 **Save or Display the DataFrame**
print(dfs.head())
dfs.to_csv("skysports_liverpool_test.csv", index=False)

print("✅ Scraping completed successfully!")

✅ Load More button found! Clicking once for testing...
✅ Load More button clicked successfully!
        Date   Time                                           Headline  \
0  3/17/2025  14:33  Merson calls for perspective after 'bad week' ...   
1  3/17/2025  11:59  Ref Watch: Carabao Cup Final - Big decisions a...   
2  3/17/2025  11:08  Van Dijk: There'll be news on my Liverpool fut...   
3  3/16/2025  20:26           LISTEN: Carabao Cup final review podcast   
4  3/16/2025  19:56       Carra: Worst cup final performance for years   

                                        Body Context  
0  Paul Merson says Liverpool have had a bad week...  
1  The Ref Watch team cast their eye over the big...  
2  LiverpoolcaptainVirgil van Dijksays there will...  
3  Mark Chapman is joined by Shay Given, Les Ferd...  
4  Sky Sports' Jamie Carragher reflecting on Live...  
✅ Scraping completed successfully!


In [35]:
import time  
from selenium import webdriver  
from selenium.webdriver.edge.service import Service as EdgeService  
from selenium.webdriver.edge.options import Options as EdgeOptions  
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  

# 1️⃣ **Set up Microsoft Edge WebDriver**
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")  # Run in headless mode (remove this if you want to see the browser)
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2️⃣ **Navigate to your target URL**
url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

# 3️⃣ **Wait for the page to load**
time.sleep(5)  # Allow JavaScript to load (you can adjust this)

iframes = driver.find_elements(By.TAG_NAME, "iframe")
print(f"🔍 Found {len(iframes)} iframes.")

if len(iframes) > 0:
    driver.switch_to.frame(iframes[0])  # Switch to first iframe
    print("✅ Switched to iframe.")

🔍 Found 18 iframes.
✅ Switched to iframe.


In [ ]:
##This is Anfield

In [63]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
import unicodedata

# Set up Microsoft Edge WebDriver
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# Function to clean and normalize text
def clean_text(text):
    text = text.replace("\u2018", "'").replace("\u2019", "'")  # Fix curly quotes
    text = text.replace("\u201C", '"').replace("\u201D", '"')  # Fix curly double quotes
    text = text.replace("\u2013", "-").replace("\u2014", "-")  # Fix dashes
    text = unicodedata.normalize("NFKD", text)  # Normalize special characters
    return text.strip()

# Target URL (Main News Page)
url = "https://www.thisisanfield.com/liverpool-fc-news/"
driver.get(url)

# Wait until the news articles load
wait = WebDriverWait(driver, 30)
try:
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h2.entry-title a")))
except:
    print("❌ Timeout: Articles not found. Trying alternative methods...")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)  # Allow JavaScript to load content

# Get the full page source after ensuring content is loaded
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Extract news headlines, URLs, and dates from the main page
articles = soup.select("article")
news_data = []

for article in articles:
    headline_element = article.select_one("h2.entry-title a")
    date_element = article.select_one("time.entry-date, time.published, span.posted-on time")
    
    if headline_element and date_element:
        headline = clean_text(headline_element.text)  # Apply text cleaning
        news_url = headline_element["href"]
        date_published = clean_text(date_element.text)
        
        # Extract time from datetime attribute and format it correctly
        time_published = date_element.get("datetime", "Unknown")
        if time_published != "Unknown":
            time_published = datetime.fromisoformat(time_published.replace("Z", "+00:00")).strftime("%H:%M:%S")
        
        print(f"Extracted: {headline} | Date: {date_published} | Time: {time_published}")
        
        # Append extracted data
        news_data.append([date_published, time_published, headline, news_url])

# Now extract the body content separately
for i, (_, _, _, news_url) in enumerate(news_data):
    driver.get(news_url)
    time.sleep(2)  # Allow time for the page to load
    
    # Get article page source
    article_html = driver.page_source
    article_soup = BeautifulSoup(article_html, "html.parser")
    
    # Extract article body content using the correct selector
    body_elements = article_soup.select("div.elementor-widget-theme-post-content p, div.elementor-widget-theme-post-content h2, div.elementor-widget-theme-post-content blockquote")
    body_content = "\n".join([clean_text(element.text) for element in body_elements if element.text.strip()])
    
    # Update the corresponding entry with body content
    news_data[i].append(body_content)

# Close the WebDriver
driver.quit()

# Convert to DataFrame
dfa = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "URL", "Body Context"])

# Save to CSV with UTF-8 encoding
dfa.to_csv("thisisanfield_news.csv", index=False, encoding="utf-8-sig")

print("✅ Scraping completed successfully! Data saved as 'thisisanfield_news.csv'")

Extracted: Jeremie Frimpong transfer links explained after meeting with Liverpool trio | Date: Mar 18, 2025 | Time: 15:48:26
Extracted: Julian Alvarez to Liverpool transfer update - Atletico Madrid respond on 'official offer' | Date: Mar 18, 2025 | Time: 13:36:56
Extracted: Liverpool's youngest-ever player now part of YouTube league aged 28 | Date: Mar 18, 2025 | Time: 12:47:40
Extracted: Conor Bradley injury update in mixed news for Liverpool - "He's frustrated" | Date: Mar 18, 2025 | Time: 11:19:53
Extracted: Robbie Fowler says "unbelievable statement" sums up Arne Slot at Liverpool | Date: Mar 18, 2025 | Time: 10:46:09
Extracted: Federico Chiesa given advantage no other Liverpool player has this month | Date: Mar 18, 2025 | Time: 10:07:31
Extracted: Luis Diaz 'transfer interest' from Saudi Arabia - "concrete possibility" to leave | Date: Mar 18, 2025 | Time: 09:45:04
Extracted: Liverpool FC international break: 20 players called up for March fixtures | Date: Mar 18, 2025 | Time: 09:

In [2]:
import time
import pandas as pd
import random
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
import unicodedata

# Set up Microsoft Edge WebDriver
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# Function to clean and normalize text
def clean_text(text):
    text = text.replace("\u2018", "'").replace("\u2019", "'")  # Fix curly quotes
    text = text.replace("\u201C", '"').replace("\u201D", '"')  # Fix curly double quotes
    text = text.replace("\u2013", "-").replace("\u2014", "-")  # Fix dashes
    text = unicodedata.normalize("NFKD", text)  # Normalize special characters
    return text.strip()

# Base URL pattern for pagination
base_url = "https://www.thisisanfield.com/liverpool-fc-news/page/{}"
num_pages = 187  # Total number of pages to scrape
news_data = []

for page in range(1, num_pages + 1):
    url = base_url.format(page)
    print(f"Scraping page {page} of {num_pages}: {url}")
    driver.get(url)
    time.sleep(random.uniform(2, 6))  # Randomized delay to avoid detection
    
    # Wait until the news articles load
    wait = WebDriverWait(driver, 120)  # Increase timeout to 120 seconds
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h2.entry-title a")))
    except:
        print("❌ Timeout: Articles not found. Trying alternative methods...")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)  # Allow JavaScript to load content
    
    # Get the full page source after ensuring content is loaded
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # Extract news headlines, URLs, and dates from the main page
    articles = soup.select("article")
    
    for article in articles:
        headline_element = article.select_one("h2.entry-title a")
        date_element = article.select_one("time.entry-date, time.published, span.posted-on time")
        
        if headline_element and date_element:
            headline = clean_text(headline_element.text)  # Apply text cleaning
            news_url = headline_element["href"]
            date_published = clean_text(date_element.text)
            
            # Extract time from datetime attribute and format it correctly
            time_published = date_element.get("datetime", "Unknown")
            if time_published != "Unknown":
                time_published = datetime.fromisoformat(time_published.replace("Z", "+00:00")).strftime("%H:%M:%S")
            
            print(f"Extracted: {headline} | Date: {date_published} | Time: {time_published}")
            
            # Append extracted data (only first 4 columns)
            news_data.append([date_published, time_published, headline, news_url])
    
    # Save backup every 50 pages (only first 4 columns before body content is added)
    if page == num_pages:  # Save only at the last page
        df_backup = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "URL"])  # Exclude Body Context
        df_backup.to_csv(f"thisisanfield_news_backup_page{page}.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Saved backup at page {page}")

# Now extract the body content separately
body_backup_interval = 200  # Save body content every 200 pages
for i, (_, _, _, news_url) in enumerate(news_data):
    if i % body_backup_interval == 0 and i > 0:
        df_body_backup = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "URL", "Body Context"])
        df_body_backup.to_csv(f"thisisanfield_news_body_backup_{i}.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Saved body content backup at article {i}")
    attempt = 0
    while attempt < 3:  # Retry up to 3 times if a page fails to load
        try:
            driver.get(news_url)
            time.sleep(random.uniform(2, 7))  # Randomized delay to avoid detection
            
            # Wait for the article content
            WebDriverWait(driver, 120).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.elementor-widget-theme-post-content"))
            )

            # Get article page source
            article_html = driver.page_source
            article_soup = BeautifulSoup(article_html, "html.parser")

            # Extract article body content
            body_elements = article_soup.select("div.elementor-widget-theme-post-content p, div.elementor-widget-theme-post-content h2, div.elementor-widget-theme-post-content blockquote")
            body_content = "\n".join([clean_text(element.text) for element in body_elements if element.text.strip()])

            # Update the corresponding entry with body content
            news_data[i].append(body_content)
            break  # Exit retry loop if successful

        except Exception as e:
            attempt += 1
            print(f"⚠️ Attempt {attempt}: Failed to load {news_url}. Retrying...")
            time.sleep(5)  # Wait before retrying
    
    if attempt == 3:
        print(f"❌ Skipping {news_url} after 3 failed attempts")
        news_data[i].append("Error: Unable to fetch content")

# Close the WebDriver
driver.quit()

# Convert to DataFrame (with all 5 columns after body content is added)
df = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "URL", "Body Context"])

# Save to CSV with UTF-8 encoding
df.to_csv("thisisanfield_news.csv", index=False, encoding="utf-8-sig")

print("✅ Scraping completed successfully! Data saved as 'thisisanfield_news.csv'")

Scraping page 1 of 187: https://www.thisisanfield.com/liverpool-fc-news/page/1
Extracted: Leipzig's 17-goal striker "100 percent" on Liverpool's radar - but is he too raw? | Date: Mar 19, 2025 | Time: 11:42:25
Extracted: Dirk Kuyt tells Darwin Nunez is up to him "whether he will play or not" | Date: Mar 19, 2025 | Time: 11:00:16
Extracted: Liverpool to wear 3 different badges on their Adidas kits next season | Date: Mar 19, 2025 | Time: 10:16:44
Extracted: Liverpool title winner to make history as first woman to play for LFC Legends | Date: Mar 19, 2025 | Time: 10:00:31
Extracted: Darwin Nunez admits international break is a chance to "forget" about Liverpool | Date: Mar 19, 2025 | Time: 09:23:10
Extracted: Title permutations, contracts & 5 more questions Liverpool FC fans are asking | Date: Mar 19, 2025 | Time: 07:00:08
Extracted: Virgil van Dijk €100m PSG offer claim rejected - Latest Liverpool FC News | Date: Mar 18, 2025 | Time: 17:02:18
Extracted: Jeremie Frimpong transfer links e

KeyboardInterrupt: 

In [22]:
import time
import pandas as pd
import random
import json
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
import unicodedata

# Set up Microsoft Edge WebDriver
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
]
edge_options.add_argument(f"--user-agent={random.choice(user_agents)}")
driver = webdriver.Edge(service=service, options=edge_options)

# Function to clean and normalize text
def clean_text(text):
    text = text.replace("\u2018", "'").replace("\u2019", "'")  # Fix curly quotes
    text = text.replace("\u201C", '"').replace("\u201D", '"')  # Fix curly double quotes
    text = text.replace("\u2013", "-").replace("\u2014", "-")  # Fix dashes
    text = unicodedata.normalize("NFKD", text)  # Normalize special characters
    return text.strip()

# Load CSV file with URLs
csv_file = "thisisanfield_news_backup_link.csv"
df = pd.read_csv(csv_file)
news_data = df.values.tolist()  # Convert DataFrame to list

start_index = 0  # Start from row 201
news_data = news_data[start_index:]

# Backup interval
body_backup_interval = 100

# Now extract the body content separately
for i, row in enumerate(news_data):
    news_url = row[3]  # URL is in the 4th column
    
    if i % body_backup_interval == 0 and i > 0:
        with open(f"thisisanfield_news_body_backup_{i}.json", "w", encoding="utf-8") as f:
            json.dump(news_data, f, indent=4, ensure_ascii=False)
        df_body_backup = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "URL", "Body Context"])
        df_body_backup.to_csv(f"thisisanfield_news_body_backup_{i}.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Saved body content backup at article {i}")
    
    attempt = 0
    while attempt < 3:
        if attempt > 0:
            print(f'🕒 Waiting before retry {attempt}...')
            time.sleep(random.uniform(10, 20))  # Exponential backoff for retries
        try:
            driver.get(news_url)
            
            time.sleep(random.uniform(1, 7))  # Increased delay to reduce request frequency
            
            # Wait for the article content
            WebDriverWait(driver, 120).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.elementor-widget-theme-post-content"))
            )

            # Get article page source
            article_html = driver.page_source
            article_soup = BeautifulSoup(article_html, "html.parser")

            # Extract article body content
            body_elements = article_soup.select("div.elementor-widget-theme-post-content p, div.elementor-widget-theme-post-content h2, div.elementor-widget-theme-post-content blockquote, div.entry-content p")
            body_content = "\n".join([clean_text(element.text) for element in body_elements if element.text.strip()])
            if not body_content.strip():
                print(f"⚠️ Warning: No body content found for {news_url}")

            # Update the corresponding entry with body content
            if len(news_data[i]) == 4:  # Ensure only appending once
                news_data[i].append(body_content)
            print(f'✅ Done status: Row {i + 1} completed.')
            print(f'🔎 First 10 words: {" ".join(body_content.split()[:10])}')
            break  # Exit retry loop if successful

        except Exception as e:
            attempt += 1
            print(f"⚠️ Attempt {attempt}: Failed to load {news_url}. Retrying...")
            time.sleep(5)  # Wait before retrying
    
    if attempt == 3:
        print(f"❌ Skipping {news_url} after 3 failed attempts")
        if len(news_data[i]) == 4:
            news_data[i].append("Error: Unable to fetch content")

# Close the WebDriver
driver.quit()

# Convert to DataFrame (with all 5 columns after body content is added)
with open("thisisanfield_news_updated.json", "w", encoding="utf-8") as f:
    json.dump(news_data, f, indent=4, ensure_ascii=False)

print("✅ Scraping completed successfully! Data saved as 'thisisanfield_news_updated.json'")

✅ Done status: Row 1 completed.
🔎 First 10 words: Bayer Leverkusen right-back Jeremie Frimpong has been linked with a
✅ Done status: Row 2 completed.
🔎 First 10 words: Julian Alvarez was subject of interest from Liverpool last summer,
✅ Done status: Row 3 completed.
🔎 First 10 words: The record for Liverpool's youngest-ever player has not been broken
✅ Done status: Row 4 completed.
🔎 First 10 words: Conor Bradley is not expected to make an imminent return
✅ Done status: Row 5 completed.
🔎 First 10 words: Former striker Robbie Fowler believes some perspective needs applying to
✅ Done status: Row 6 completed.
🔎 First 10 words: Federico Chiesa is in a position unique to the rest
✅ Done status: Row 7 completed.
🔎 First 10 words: Luis Diaz is expected to attract interest in the summer
✅ Done status: Row 8 completed.
🔎 First 10 words: Liverpool will have 20 senior players on international duty in
✅ Done status: Row 9 completed.
🔎 First 10 words: Liverpool have reportedly been in contact with

KeyboardInterrupt: 

In [34]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

# Function to clean and normalize text
def clean_text(text):
    return text.strip()

# Load CSV file
csv_file = "thisisanfield_news_backup_link.csv"
df = pd.read_csv(csv_file)

# Ensure "Body Context" column exists
if "Body Context" not in df.columns:
    df["Body Context"] = "No Content"

# Loop through each row in CSV
for index, row in df.iterrows():
    url = row["URL"]
    print(f"🔍 Scraping: {url}")

    try:
        # Send HTTP request to fetch page content
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)

        # Check if request was successful
        if response.status_code != 200:
            print(f"⚠️ Error {response.status_code}: Skipping {url}")
            df.at[index, "Body Context"] = f"Error {response.status_code}"
            continue

        # Parse HTML content
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract article body content
        body_elements = soup.select("div.elementor-widget-theme-post-content p, div.entry-content p")
        body_content = "\n".join([clean_text(element.text) for element in body_elements if element.text.strip()])

        # Check if content is found
        if not body_content:
            print(f"⚠️ Warning: No body content found for {url}")
            df.at[index, "Body Context"] = "No Content"
        else:
            df.at[index, "Body Context"] = body_content[:500]  # Limit content to 500 characters for preview

        # ✅ Print the first few words of the scraped content
        print(f"✅ Successfully Scraped Row {index + 1}:")
        print(f"🔎 First 10 words: {' '.join(body_content.split()[:10])}...\n")

        # Add random delay to prevent blocking
        time.sleep(random.uniform(2, 5))

    except Exception as e:
        print(f"❌ Error: {e}")
        df.at[index, "Body Context"] = "Scraping Error"

    # Save temporary progress every 10 rows
    if (index + 1) % 10 == 0:
        df.to_csv("thisisanfield_news_temp.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Temporary save at row {index + 1}")

# Save final data to CSV
df.to_csv("thisisanfield_news_updated.csv", index=False, encoding="utf-8-sig")
print("🎉 Scraping Completed! Data saved in 'thisisanfield_news_updated.csv'")

🔍 Scraping: https://www.thisisanfield.com/2025/03/jeremie-frimpong-transfer-links-explained-after-meeting-with-liverpool-trio/
✅ Successfully Scraped Row 1:
🔎 First 10 words: Bayer Leverkusen right-back Jeremie Frimpong has been linked with a...

🔍 Scraping: https://www.thisisanfield.com/2025/03/julian-alvarez-to-liverpool-transfer-update-atletico-madrid-respond-on-official-offer/
✅ Successfully Scraped Row 2:
🔎 First 10 words: Julian Alvarez was subject of interest from Liverpool last summer,...

🔍 Scraping: https://www.thisisanfield.com/2025/03/liverpools-youngest-ever-player-now-part-of-youtube-league-aged-28/
✅ Successfully Scraped Row 3:
🔎 First 10 words: The record for Liverpool’s youngest-ever player has not been broken...

🔍 Scraping: https://www.thisisanfield.com/2025/03/conor-bradley-injury-update-in-mixed-news-for-liverpool-hes-frustrated/
✅ Successfully Scraped Row 4:
🔎 First 10 words: Conor Bradley is not expected to make an imminent return...

🔍 Scraping: https://www.this